<a href="https://colab.research.google.com/github/soerenetler/qg_with_attention/blob/master/tedq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/amore-upf/ted-q.git
!git clone https://github.com/MurathanKurfali/Ted-MDB-Annotations.git

fatal: destination path 'ted-q' already exists and is not an empty directory.
fatal: destination path 'Ted-MDB-Annotations' already exists and is not an empty directory.


In [11]:
import pandas as pd
df = pd.read_csv("ted-q/TED-Q_elicitation.csv", index_col="id")
df.head()

,excerpt_number,chunk_number,worker,type,content,wh_type,answered,highlight,prior_question,best_answer,coherence,naturalness,comment,relatedness,source,chunk_start,chunk_end,highlight_start,highlight_end
id,,,,,,,,,,,,,,,,,,,
1565638102.f97d405b607c9aca212d884d340cb014.10.0,1,0,Solomon,question,Why did she want to run away?,why,2.0,one day run away,NaN,1565638102.f97d405b607c9aca212d884d340cb014.10.1,NaN,NaN,NaN,2.833333,Ted-MDB-Annotations/English/raw/01/talk_2009_e...,61.0,239.0,105.0,121.0
1565638102.f97d405b607c9aca212d884d340cb014.10.1,1,1,Solomon,non-answer,NaN,NaN,2.0,NaN,['1565638102.f97d405b607c9aca212d884d340cb014....,NaN,NaN,NaN,NaN,NaN,Ted-MDB-Annotations/English/raw/01/talk_2009_e...,239.0,465.0,NaN,NaN
1565638102.f97d405b607c9aca212d884d340cb014.10.3,1,1,Solomon,question,Is your home life bad?,aux,1.0,a deep restlessness,NaN,1565638102.f97d405b607c9aca212d884d340cb014.10.4,NaN,NaN,NaN,0.625000,Ted-MDB-Annotations/English/raw/01/talk_2009_e...,239.0,465.0,249.0,268.0
1565638102.f97d405b607c9aca212d884d340cb014.10.4,1,2,Solomon,non-answer,NaN,NaN,1.0,NaN,['1565638102.f97d405b607c9aca212d884d340cb014....,NaN,NaN,NaN,NaN,NaN,Ted-MDB-Annotations/English/raw/01/talk_2009_e...,465.0,755.0,NaN,NaN
1565638102.f97d405b607c9aca212d884d340cb014.10.5,1,2,Solomon,non-answer,NaN,NaN,2.0,NaN,['1565638102.f97d405b607c9aca212d884d340cb014....,NaN,NaN,NaN,NaN,NaN,Ted-MDB-Annotations/English/raw/01/talk_2009_e...,465.0,755.0,NaN,NaN


In [12]:
import plotly.express as px
px.histogram(df[df["type"] == "question"], "wh_type", histnorm="percent")

In [13]:
len(df[df["type"] == "question"])

2412

In [14]:
df[df["wh_type"] == "unknown"]

,excerpt_number,chunk_number,worker,type,content,wh_type,answered,highlight,prior_question,best_answer,coherence,naturalness,comment,relatedness,source,chunk_start,chunk_end,highlight_start,highlight_end
id,,,,,,,,,,,,,,,,,,,
1565638102.f97d405b607c9aca212d884d340cb014.62.0,5,0,Solomon,question,What's an institutional investor?,unknown,1.0,institutional investors,NaN,1565638102.f97d405b607c9aca212d884d340cb014.62.1,NaN,NaN,NaN,1.500000,Ted-MDB-Annotations/English/raw/01/talk_1927_e...,7331.0,7496.0,7335.0,7358.0
1565638102.f97d405b607c9aca212d884d340cb014.62.6,5,2,Solomon,question,What's a global bond market?,unknown,1.0,global bond market,NaN,1565638102.f97d405b607c9aca212d884d340cb014.62.7,NaN,NaN,NaN,1.071429,Ted-MDB-Annotations/English/raw/01/talk_1927_e...,7578.0,7645.0,7582.0,7600.0
1565638102.f97d405b607c9aca212d884d340cb014.62.12,5,4,Solomon,question,What's ESG?,unknown,1.0,ESG,NaN,1565638102.f97d405b607c9aca212d884d340cb014.62.13,NaN,NaN,NaN,1.500000,Ted-MDB-Annotations/English/raw/01/talk_1927_e...,7771.0,7942.0,7921.0,7924.0
1565645239.cd396b089d082077d9612031afc8872c.15.15,1,5,Isidro,question,Have we attempted it before?,unknown,2.0,Why haven't we done that?,NaN,1565645239.cd396b089d082077d9612031afc8872c.15.16,NaN,NaN,NaN,1.102679,Ted-MDB-Annotations/English/raw/01/talk_1976_e...,986.0,1030.0,986.0,1011.0
1565645239.cd396b089d082077d9612031afc8872c.40.15,2,5,Isidro,question,What's the most popular kind of music in Rio D...,unknown,2.0,different kinds of music,NaN,1565645239.cd396b089d082077d9612031afc8872c.40.16,NaN,NaN,NaN,0.666667,Ted-MDB-Annotations/English/raw/01/talk_2150_e...,3571.0,3725.0,3653.0,3677.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1566188724.07978312887e4b92832085e5e97d8127.64.3,5,1,Cayla,question,which ones are acceptable expected risks?,unknown,1.0,'There are risks and costs to a program of action,NaN,1566188724.07978312887e4b92832085e5e97d8127.64.4,NaN,NaN,NaN,1.500000,Ted-MDB-Annotations/English/raw/01/talk_1927_e...,8406.0,8591.0,8540.0,8589.0
1566188724.07978312887e4b92832085e5e97d8127.64.9,5,3,Cayla,question,sre you saying there is flaw in the study?,unknown,2.0,better than the odds,NaN,1566188724.07978312887e4b92832085e5e97d8127.64.10,NaN,NaN,NaN,0.744444,Ted-MDB-Annotations/English/raw/01/talk_1927_e...,8791.0,8957.0,8835.0,8855.0
1566188724.07978312887e4b92832085e5e97d8127.64.12,5,4,Cayla,question,"Insurance is one way to vaid disaster, but wha...",unknown,1.0,buy insurance to protect ourselves financially,NaN,1566188724.07978312887e4b92832085e5e97d8127.64.13,NaN,NaN,NaN,1.821429,Ted-MDB-Annotations/English/raw/01/talk_1927_e...,8957.0,9075.0,8992.0,9038.0


In [15]:
df["source"].unique()

array(['Ted-MDB-Annotations/English/raw/01/talk_2009_en.txt',
       'Ted-MDB-Annotations/English/raw/01/talk_2150_en_inter.txt',
       'Ted-MDB-Annotations/English/raw/01/talk_1927_en.txt',
       'Ted-MDB-Annotations/English/raw/01/talk_1976_en.txt',
       'Ted-MDB-Annotations/English/raw/01/talk_1978_en.txt',
       'Ted-MDB-Annotations/English/raw/01/talk_1971_en.txt'],
      dtype=object)

In [7]:
!pip install --quiet -U spacy[cuda110,transformers,lookups]

In [8]:
!python -m spacy download en_core_web_sm --quiet
!python -m spacy download en_core_web_trf --quiet
!python -m spacy download en_core_web_lg --quiet

2021-06-13 09:19:11.777142: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
2021-06-13 09:19:18.976227: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')
2021-06-13 09:19:26.122308: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [9]:
import spacy
nlp = spacy.load("en_core_web_trf")
text_dict = {}

for file in df["source"].unique():
  with open(file) as f:
    text = f.read()
    text_dict[file] = nlp(text)

In [39]:
tedq_qg_dataset = {
    "answer_sentence_token": [],
    "answer": [],
    "answer_token": [],
    "question_token": []
}

for i, question_row in df[df["type"] == "question"].iterrows():
  answer_id = question_row["best_answer"]
  answer_row = df.loc[answer_id]
  if answer_row["type"] != "non-answer":
    source_text = answer_row["source"]
    chunk_start = answer_row["chunk_start"]
    chunk_end = answer_row["chunk_end"]
    highlight_start= answer_row["highlight_start"]
    highlight_end= answer_row["highlight_end"]

    question = question_row["content"]
    tedq_qg_dataset["question_token"].append([t.text for t in nlp(question)])
    tedq_qg_dataset["answer"].append(text_dict[source_text].char_span(highlight_start,highlight_end, alignment_mode="expand").text)
    tedq_qg_dataset["answer_token"].append([t.text for t in text_dict[source_text].char_span(highlight_start,highlight_end, alignment_mode="expand")])
    tedq_qg_dataset["answer_sentence_token"].append([t.text for t in text_dict[source_text].char_span(highlight_start,highlight_end, alignment_mode="expand").sent])

In [44]:
tedq_qg_df = pd.DataFrame(tedq_qg_dataset)
tedq_qg_df.to_csv("ted_q.csv")

In [43]:
tedq_qg_df

,answer_sentence_token,answer,answer_token,question_token
0,"[But, no, other, experience, has, felt, as, tr...",lives of fellow wanderers,"[lives, of, fellow, wanderers]","[Do, you, want, to, live, on, the, road, ?]"
1,"[\n\n, When, we, think, about, mapping, cities...",but there's other ways to think about mapping ...,"[but, there, 's, other, ways, to, think, about...","[Is, there, more, to, map, ?]"
2,"[This, is, a, map, of, the, relationships, bet...",relationships,[relationships],"[What, is, the, new, map, ?]"
3,"[This, is, a, map, of, the, relationships, bet...",color represents a community within the network,"[color, represents, a, community, within, the,...","[How, can, you, map, people, ?]"
4,"[If, we, make, the, edges, of, those, petals, ...",now we have a great shadow,"[now, we, have, a, great, shadow]","[Why, do, you, call, it, a, flower, pedal, sha..."
...,...,...,...,...
1102,"[She, first, hit, a, seven, ,, I, remember, ,,...",two tens,"[two, tens]","[Did, the, archer, hit, the, ten, ring, ?]"
1103,"[She, first, hit, a, seven, ,, I, remember, ,,...",two tens,"[two, tens]","[Did, the, archer, hit, the, tiny, ring, ?]"
1104,"[I, like, it, because, it, pokes, fun, at, bot...",because it pokes fun at both sides,"[because, it, pokes, fun, at, both, sides]","[What, does, the, person, like, ?]"
1105,"[I, like, it, because, it, pokes, fun, at, bot...",pokes fun at both sides of the climate change,"[pokes, fun, at, both, sides, of, the, climate...","[What, do, they, both, like, ?]"
